In [ ]:
import pandas as pd
import numpy as np
import pickle
import re

In [ ]:
# 2. Carreguem dades
df = pd.read_pickle("data/datosFinal.pkl")

In [ ]:
df

### Preprocessing

1. Company

In [ ]:
condicio = (
    df["Company"].str.strip().str.upper().str.endswith("S.L") |
    df["Company"].str.strip().str.upper().str.endswith("SL") |
    df["Company"].str.strip().str.upper().str.endswith("S.L.") |
    df["Company"].str.upper().str.contains("SOCIEDAD LIMITADA")
)

df['es_sociedad_limitada'] = np.where(condicio, 1, 0)

df.es_sociedad_limitada.value_counts()

2. Names

In [ ]:
(df["Company"] == df["Name"]).value_counts()
df = df.drop(columns=["Company"])

3. Investment

In [ ]:
df["Investment"] = (
    df["capital_prev"]
    .astype(str)
    .str.extract(r'([\d.,]+M\s?€)')[0]   # agafa només la part numèrica amb M i €
    .str.replace("€", "", regex=False)
    .str.replace("M", "", regex=False)
    .str.replace("m", "", regex=False)
    .str.replace(",", ".", regex=False)       # canvia coma per punt decimal
    .str.replace(" ", "", regex=False)        # elimina espais
)

# 2. Conversió a numèric (float), posant NaN si hi ha errors
df["Investment"] = pd.to_numeric(df["Investment"], errors="coerce")

# 3. Substitueix 0 per NaN
df["Investment"] = df["Investment"].replace(0.0, np.nan)

# per tenir 0 en comptes de nan
df["Investment"] = df["Investment"].fillna(0)


df.Investment

4. Investor

In [ ]:
df.Investors = df.investors
df = df.drop(columns=['investors'])

In [ ]:
all_investors = []

for group in df['Investors'].dropna().tolist():
    splitted = [inv.strip() for inv in group.split(',')]
    all_investors.extend(splitted)

investor_series = pd.Series(all_investors)

# Comptar freqüències
freq_table = investor_series.value_counts().reset_index()

# Renombrar columnes
freq_table.columns = ['Investor', 'Frequency']

# Mostrar la taula
print(freq_table)

In [ ]:
import matplotlib.pyplot as plt

# Suposem que tens aquesta taula:
# freq_table.columns = ['Investor', 'Frequency']

plt.figure(figsize=(8, 5))
plt.hist(freq_table['Frequency'], bins=range(1, freq_table['Frequency'].max() + 2), edgecolor='black')
plt.title('Histograma de Freqüència dels Inversors')
plt.xlabel('Nombre de vegades que un inversor apareix')
plt.ylabel('Nombre d\'inversors')
plt.xticks(range(1, freq_table['Frequency'].max() + 1))
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()


In [ ]:
#De moment no faig la columna investors perquè hi ha molts pocs que es repeteixen i encara haig de pensar com fer per saber si són persones o no

5. Date

In [ ]:
df.Date = df['capital_prev'].str.extract(r'\((.*?)\)')

In [ ]:
df = df.drop(columns=['capital_prev'])

In [ ]:
df

In [ ]:
df["Mes Inversion"] = [mes.split(" ")[0] for mes in df["Date"]]
df["Year Inversion"] = [mes.split(" ")[1] for mes in df["Date"]]
## Eliminar variable Date

In [ ]:
import pandas as pd
from datetime import datetime

# Suposem que tens un DataFrame anomenat df

# 1. Crear una data a partir de les columnes "Mes inversion" i "Year inversion"
# Convertim el nom del mes a número
df['Mes_num'] = pd.to_datetime(df['Mes Inversion'], format='%B').dt.month

# Crear una columna amb la data completa
df['Data_inversion'] = pd.to_datetime(dict(year=df['Year Inversion'], month=df['Mes_num'], day=1))

# 2. Definir la data actual (Juny 2025)
data_actual = datetime(2025, 6, 3)

# 3. Calcular la diferència en mesos
diferencia_mesos = (data_actual.year - df['Data_inversion'].dt.year) * 12 + (data_actual.month - df['Data_inversion'].dt.month)

# 4. Convertir a anys amb decimals
df['Antiguitat'] = diferencia_mesos / 12
df['Antiguitat']


6. Adreça

Al final no la fem

7. Industries

In [ ]:
# Elimina salts de línia, tabulacions i espais extra
df['Industries'] = df['Industries'].astype(str) \
    .str.replace(r'[\r\n\t]', '', regex=True) \
    .str.replace(r'\s{2,}', ' ', regex=True) \
    .str.strip()

In [ ]:
# Inicialitzar llista buida
all_industries = []

# Iterar per cada fila
for item in df['Industries'].dropna():
    industries = [i.strip() for i in item.split(',')]
    all_industries.extend(industries)

industries_series = pd.Series(all_industries)
freq_table = industries_series.value_counts().reset_index()
freq_table.columns = ['Industry', 'Frequency']
Industries = freq_table.Industry
print(freq_table)

#Veiem que només hi ha 24 industries, podem fer columnes binàries per a cada indústria

In [ ]:
Ind = df[["Name","Industries"]]
M = pd.DataFrame(columns = [Industries])
for i in range(len(Ind)):
    a = Ind.Industries[i]
    a_clean = [i.strip() for i in a.split(',')]
    binary_vector = [1 if industry in a_clean else 0 for industry in Industries]
    M.loc[i] = binary_vector

In [ ]:
# 1. Converteix el MultiIndex en columnes normals
M.columns = [col[0] for col in M.columns]

# 2. Renombra cada columna com "Ind - nom"
M.columns = [f"Ind {col}" for col in M.columns]
M.columns

In [ ]:
df_combinat = pd.concat([Ind, M], axis=1)

In [ ]:
#df = pd.concat([df, M], axis=1)

#En el cas de voler ajuntar les matrius

8. Technologies

In [ ]:
# Elimina salts de línia, tabulacions i espais extra
df['Technologies'] = df['Technologies'].astype(str) \
    .str.replace(r'[\r\n\t]', '', regex=True) \
    .str.replace(r'\s{2,}', ' ', regex=True) \
    .str.strip()
df.Technologies

In [ ]:
# Inicialitzar llista buida
all_technologies = []

# Iterar per cada fila
for item in df['Technologies'].dropna():
    technologies = [i.strip() for i in item.split(',')]
    all_technologies.extend(technologies)

# Crear una sèrie i taula de freqüències
technologies_series = pd.Series(all_technologies)
tech_freq_table = technologies_series.value_counts().reset_index()
tech_freq_table.columns = ['Technology', 'Frequency']

# Guardem la llista única de tecnologies
Technologies = tech_freq_table.Technology
print(tech_freq_table)

# Ara ja pots fer columnes binàries per cada tecnologia

In [ ]:
Tech = df[["Name","Technologies"]]
T = pd.DataFrame(columns = [Technologies])
for i in range(len(Tech)):
    a = Tech.Technologies[i]
    a_clean = [i.strip() for i in a.split(',')]
    binary_vector = [1 if technology in a_clean else 0 for technology in Technologies]
    T.loc[i] = binary_vector

In [ ]:
# 1. Converteix el MultiIndex en columnes normals
T.columns = [col[0] for col in T.columns]

# 2. Renombra cada columna com "Ind - nom"
T.columns = [f"Tech {col}" for col in T.columns]
T.columns

In [ ]:
df_combinat = pd.concat([df_combinat, T], axis=1)

In [ ]:
#df = pd.concat([df, T], axis=1)

#En el cas de voler ajuntar les matrius

9. Categoria

In [ ]:
df = df.drop("Category", axis = 1)

10. Other fields

In [ ]:
# Elimina salts de línia, tabulacions i espais extra
df['Other fields'] = df['Other fields'].astype(str) \
    .str.replace(r'[\r\n\t]', '', regex=True) \
    .str.replace(r'\s{2,}', ' ', regex=True) \
    .str.strip()
df["Other fields"]

In [ ]:
# Inicialitzar llista buida
all_fields = []

# Iterar per cada fila
for item in df['Other fields'].dropna():
    fields = [i.strip() for i in item.split(',')]
    all_fields.extend(fields)

# Crear una sèrie i taula de freqüències
fields_series = pd.Series(all_fields)
fields_freq_table = fields_series.value_counts().reset_index()
fields_freq_table.columns = ['Field', 'Frequency']

# Guardem la llista única de camps
OtherFields = fields_freq_table.Field
print(fields_freq_table)

In [ ]:
OFields = df[["Name","Other fields"]]
OF = pd.DataFrame(columns = [OtherFields])
for i in range(len(OFields)):
    a = OFields["Other fields"][i]
    a_clean = [i.strip() for i in a.split(',')]
    binary_vector = [1 if otherfield in a_clean else 0 for otherfield in OtherFields]
    OF.loc[i] = binary_vector

In [ ]:
# 1. Converteix el MultiIndex en columnes normals
OF.columns = [col[0] for col in OF.columns]

# 2. Renombra cada columna com "Ind - nom"
OF.columns = [f"OF {col}" for col in OF.columns]
OF.columns

In [ ]:
df_combinat = pd.concat([df_combinat, OF], axis=1)

In [ ]:
#df = pd.concat([df, OF], axis=1)

#En el cas de voler ajuntar les matrius

11. Funding stage

In [ ]:
#No te sentit aquesta variable ja que ja tenim la quantitat exacte
df = df.drop("Funding stage", axis = 1)

12. Founded

In [ ]:
df["Founded"] = 2025-df["Founded"].astype(float)
df_clean = df.dropna(subset=['Founded'])

plt.figure(figsize=(10, 6))
plt.hist(df_clean['Founded'], bins=20, edgecolor='black')
plt.title('Histograma de l\'any de fundació')
plt.xlabel('Any de fundació')
plt.ylabel('Nombre de startups')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

13. Employees

In [ ]:
dummies = pd.get_dummies(df['Employees'], prefix='Emp').astype(int)
dummies.columns = dummies.columns.str.replace('_', ' ', regex=False)
df_combinat = pd.concat([df_combinat, dummies], axis=1)

14. Business model

In [ ]:
# Elimina salts de línia, tabulacions i espais extra
df['Business model'] = df['Business model'].astype(str) \
    .str.replace(r'[\r\n\t]', '', regex=True) \
    .str.replace(r'\s{2,}', ' ', regex=True) \
    .str.strip()
df["Business model"]

In [ ]:
# Inicialitzar llista buida
all_models = []

# Iterar per cada fila
for item in df['Business model'].dropna():
    models = [i.strip() for i in item.split(',')]
    all_models.extend(models)

# Crear una sèrie i taula de freqüències
models_series = pd.Series(all_models)
models_freq_table = models_series.value_counts().reset_index()
models_freq_table.columns = ['Business model', 'Frequency']

# Guardar la llista única de models
BusinessModels = models_freq_table['Business model']

print(models_freq_table)

In [ ]:
BModels = df[["Name","Business model"]]
BM = pd.DataFrame(columns = [BusinessModels])
for i in range(len(BModels)):
    a = BModels["Business model"][i]
    a_clean = [i.strip() for i in a.split(',')]
    binary_vector = [1 if businessmodel in a_clean else 0 for businessmodel in BusinessModels]
    BM.loc[i] = binary_vector

In [ ]:
# 1. Converteix el MultiIndex en columnes normals
BM.columns = [col[0] for col in BM.columns]

# 2. Renombra cada columna com "Ind - nom"
BM.columns = [f"BM {col}" for col in BM.columns]
BM.columns

In [ ]:
print(models_freq_table)

In [ ]:
sums = BM.sum()
print(sums)

In [ ]:
df_combinat = pd.concat([df_combinat, BM], axis=1)

In [ ]:
#df = pd.concat([df, BM], axis=1)

#En el cas de voler ajuntar les matrius

15. Target

In [ ]:
df['Target'] = df['Target'].astype(str) \
    .str.replace(r'[\r\n\t]', '', regex=True) \
    .str.replace(r'\s{2,}', ' ', regex=True) \
    .str.strip()
df["Target"]

In [ ]:
# Inicialitzar llista buida
all_targets = []

# Iterar per cada fila
for item in df['Target'].dropna():
    targets = [i.strip() for i in item.split(',')]
    all_targets.extend(targets)

# Crear una sèrie i taula de freqüències
targets_series = pd.Series(all_targets)
targets_freq_table = targets_series.value_counts().reset_index()
targets_freq_table.columns = ['Target', 'Frequency']

# Guardar la llista única de targets
Targets = targets_freq_table['Target']

print(targets_freq_table)

In [ ]:
Targ = df[["Name","Target"]]
Ta = pd.DataFrame(columns = [Targets])
for i in range(len(Targ)):
    a = Targ["Target"][i]
    a_clean = [i.strip() for i in a.split(',')]
    binary_vector = [1 if targets in a_clean else 0 for targets in Targets]
    Ta.loc[i] = binary_vector

In [ ]:
# 1. Converteix el MultiIndex en columnes normals
Ta.columns = [col[0] for col in Ta.columns]

# 2. Renombra cada columna com "Ind - nom"
Ta.columns = [f"Targ {col}" for col in Ta.columns]
Ta.columns

In [ ]:
df_combinat = pd.concat([df_combinat, Ta], axis=1)

In [ ]:
#df = pd.concat([df, Ta], axis=1)

#En el cas de voler ajuntar les matrius

16. Spinoff participants

In [ ]:
df["Spinoff participants"] = df["Spinoff participants"].fillna("").astype(str) \
    .str.replace(r'\(.*?\)', '', regex=True) \
    .str.replace(r'\n', '', regex=True) \
    .str.replace(r'/', '', regex=True) \
    .str.replace(r'\)', '', regex=True) \
    .str.replace(';', ',', regex=False) \
    .str.replace('Fundació Institut d’Investigació i Innovació Parc Tauli', ',', regex=False) \
    .str.replace(' ,', ',', regex=False) \
    .str.strip()
print(df["Spinoff participants"].iloc[35:45])

In [ ]:
# Inicialitzar llista buida
all_participants = []

# Iterar per cada fila
for item in df['Spinoff participants'].dropna():
    participants = [i.strip() for i in item.split(',')]
    all_participants.extend(participants)

# Crear una sèrie i taula de freqüències
participants_series = pd.Series(all_participants)
participants_freq_table = participants_series.value_counts().reset_index()
participants_freq_table.columns = ['Spinoff Participant', 'Frequency']

#Esborro la fila 9 per un error
participants_freq_table = participants_freq_table.drop(index=9).reset_index(drop=True)

# Guardar la llista única de participants
SpinoffParticipants = participants_freq_table['Spinoff Participant']

# Mostrar la taula de freqüències
print(participants_freq_table)


In [ ]:
Spin = df[["Name","Spinoff participants"]]
S = pd.DataFrame(columns = [SpinoffParticipants])
for i in range(len(Spin)):
    a = Spin["Spinoff participants"][i]
    a_clean = [i.strip() for i in a.split(',')]
    binary_vector = [1 if spinoffparticipants in a_clean else 0 for spinoffparticipants in SpinoffParticipants]
    S.loc[i] = binary_vector

In [ ]:
# 1. Converteix el MultiIndex en columnes normals
S.columns = [col[0] for col in S.columns]

# 2. Renombra cada columna com "Ind - nom"
S.columns = [f"Spin {col}" for col in S.columns]
S.columns

In [ ]:
df_combinat = pd.concat([df_combinat, S], axis=1)

In [ ]:
#df = pd.concat([df, S], axis=1)

#En el cas de voler ajuntar les matrius

17. Quantiat d'inversions previes

In [ ]:
df["Num_Inversio"] = df.groupby("Name").cumcount(ascending=False) + 1
print(df[["Name","Num_Inversio"]].head(15))

Para la variable capita_prev contrastar con las variables Investor y Date si son iguales sino se deberá de trabajar para tener 2 variables independientes. 

In [ ]:
df = df[['Name', 'Startup', 'Description', 'Investment', 'Num_Inversio', 'Founded', 'es_sociedad_limitada']]

In [ ]:
#df = df.drop(columns=["URL", "Date","Investors","Industries","Technologies","Other fields","Business model","Target","Spinoff participants"])

In [ ]:
df_combinat = df_combinat.drop(columns=["Name","Industries"])

In [ ]:
df = pd.concat([df, df_combinat], axis=1)

In [ ]:
duplicats = df.columns[df.columns.duplicated()].tolist()
print(duplicats)

In [ ]:
df.iloc[:,3:10].info()

## Guardamos la base de datos 

In [ ]:
df.to_pickle(path="data/datos_preprocesados.pkl")
df.to_csv(path_or_buf="data/datos_preprocesados.csv")